1.Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости.
Для парсинга использовать XPath. Структура данных должна содержать:

название источника;

наименование новости;

ссылку на новость;

дата публикации.
    
2.Сложить собранные новости в БД
Минимум один сайт, максимум - все три

In [1]:
import hashlib
from lxml import html
import requests
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

In [ ]:
client = MongoClient('localhost', 27017)
db = client['lesson_4_DB']
bbc_articles = db.bbc_articles

url = 'https://www.bbc.com/russian'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/98.0.4758.102 Safari/537.36'}
response = requests.get(url, headers=header)
dom = html.fromstring(response.text)


def add_to_database():
    articles = dom.xpath("//li[@class='e57qer20 bbc-lpu9rr eom0ln51']")
    for article in articles:
        articles_data = {}
        name = article.xpath(".//h3[@class='bbc-t81ytv e1tfxkuo2']/a[@href]/span[@id]/text()")
        link = article.xpath(".//h3[@class='bbc-t81ytv e1tfxkuo2']/a[@href]/@href")
        time = article.xpath(".//div[@class='bbc-1sk5sm2 e19k1v2h0']/time[@datetime]/text()")

        name_pre_encode = name[0].encode()
        name_encode = hashlib.sha256(name_pre_encode)
        name_hex = name_encode.hexdigest()

        articles_data['_id'] = name_hex
        articles_data['name'] = name
        articles_data['link'] = f"{url}{link[0]}"
        articles_data['time'] = time

        try:
            bbc_articles.insert_one(articles_data)
        except DuplicateKeyError:
            print(f"Document already exists")


add_to_database()

In [ ]:
url = 'https://yandex.ru/news/'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                        'Chrome/98.0.4758.102 Safari/537.36'}
response = requests.get(url, headers=header)
dom = html.fromstring(response.text)

client = MongoClient('localhost', 27017)
db = client['lesson_4_DB']
yandex_articles = db.yandex_articles


def add_to_db():
    articles = dom.xpath("//div[@class='mg-grid__col mg-grid__col_xs_6']")
    for article in articles:
        articles_data = {}

        name = article.xpath(".//a[@class='mg-card__link']/text()")
        source = article.xpath(".//a[@class='mg-card__source-link']/text()")
        time = article.xpath(".//span[@class='mg-card-source__time']/text()")
        link = article.xpath(".//a[@class='mg-card__link']/@href")

        name_pre_encode = name[0].encode()
        name_encode = hashlib.sha256(name_pre_encode)
        name_hex = name_encode.hexdigest()

        articles_data['_id'] = name_hex
        articles_data['name'] = name
        articles_data['link'] = link
        articles_data['time'] = time
        articles_data['source'] = source

        try:
            yandex_articles.insert_one(articles_data)
        except DuplicateKeyError:
            print(f"Document already exists")


add_to_db()